# Interactive version of biv-me
This jupyter notebook contains an identical implementation of biv-me to the main.py. The only difference is that it can be run step-by-step, allowing you to make manual corrections if and when needed to both the view selection and segmentation steps, which can be prone to occasional errors.

The only thing you will need on top of the regular biv-me environment is to download a Python IDE which can run Jupyter notebooks (e.g. Visual Studio Code), and install the ipython kernel as below.

```
conda install -n bivme311 ipykernel --update-deps --force-reinstall
```

When you run the notebook, you will be prompted to select the kernel to use. Make sure to select your bivme311 environment.

### Troubleshooting model performance
Though we are confident in the robustness of our deep learning models, they may not work perfectly for your data. If you find that the segmentation or view selection models perform poorly for your data, reach out to us at [joshua.dillon@auckland.ac.nz](joshua.dillon@auckland.ac.nz) or [charlene.1.mauger@kcl.ac.uk](charlene.1.mauger@kcl.ac.uk) and let us know what kind of data you are using. We are actively developing these models and always looking for ways to enhance their generalisability across vendors, protocols, centres, and patient demographics.


In [1]:
# Import libraries
import os,sys
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch
import tomli
from pathlib import Path
import shutil
import time
import datetime
from loguru import logger

import warnings
warnings.filterwarnings('ignore')

In [2]:
# This allows for modules to be reloaded automatically if you make any changes to those files.
%load_ext autoreload
%autoreload 2

In [3]:
# Import modules
from bivme.preprocessing.dicom.extract_cines import extract_cines
from bivme.preprocessing.dicom.select_views import select_views
from bivme.preprocessing.dicom.segment_views import segment_views
from bivme.preprocessing.dicom.correct_phase_mismatch import correct_phase_mismatch
from bivme.preprocessing.dicom.generate_contours import generate_contours
from bivme.preprocessing.dicom.export_guidepoints import export_guidepoints
from bivme.plotting.plot_guidepoints import generate_html 
from main import run_fitting

In [4]:
# Set up logging
log_level = "DEBUG"
log_format = "<green>{time:YYYY-MM-DD HH:mm:ss.SSS zz}</green> | <level>{level: <8}</level> | <yellow>Line {line: >4} ({file}):</yellow> <b>{message}</b>"

In [5]:
# Check if GPU is available (torch)
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    logger.warning('No GPU available. Using CPU instead. This may be very slow!')
    
logger.info(f"Using device: {device}")

2025-08-29 16:08:09.638 | INFO     | __main__:<module>:10 - Using device: cuda


# Config file
As with the main.py, you should customise the template config file in `src/bivme/configs/config.toml` for your specific batch and use case. This config file will be read in and the inputs used for the remainder of this pipeline.

In [6]:
config_file = 'configs/config.toml' # Path to the config file

In [7]:
# Load config
with open(config_file, mode="rb") as fp:
    logger.info(f'Loading config file: {config_file}')
    config = tomli.load(fp)

# TOML Schema Validation
# This needs to be done to make sure the config file is valid.
# Going forward, we will trust you that all of the inputs in the config file are correct, unlike the main.py which does not trust you.

match config:
    case {
        "modules": {"preprocessing": bool(), "fitting": bool()},

        "logging": {"show_detailed_logging": bool(), "generate_log_file": bool()},

        "plotting": {"generate_plots_preprocessing": bool(), "generate_plots_fitting": bool(), "include_images": bool(), "export_images": bool()},

        "input_pp": {"source": str(),
                    "batch_ID": str(),
                    "analyst_id": str(),
                    "processing": str(),
                    "states": str()
                    },
        "view-selection": {"option": str(), "correct_mode": str()},
        "contouring": {"smooth_landmarks": bool()},
        "output_pp": {"overwrite": bool(), "output_directory": str()},

        "input_fitting": {"gp_directory": str(),
                    "gp_suffix": str(),
                    "si_suffix": str(),
                    },
        "breathhold_correction": {"shifting": str(), "ed_frame": int()},
        "gp_processing": {"sampling": int(), "num_of_phantom_points_av": int(), "num_of_phantom_points_mv": int(), "num_of_phantom_points_tv": int(), "num_of_phantom_points_pv": int()},
        "multiprocessing": {"workers": int()},
        "fitting_weights": {"guide_points": float(), "convex_problem": float(), "transmural": float()},
        "output_fitting": {"output_directory": str(), "output_meshes": list(), "closed_mesh": bool(),   "export_control_mesh": bool(), "mesh_format": str(),  "overwrite": bool()},
    }:
        pass
    case _:
            raise ValueError(f"Invalid configuration: {config}")

2025-08-29 16:08:10.027 | INFO     | __main__:<module>:3 - Loading config file: configs/config.toml


# Start of preprocessing
This code reads in DICOM files and generates GPFiles for personalised biventricular mesh fitting.

## Step 0: Set up directories
All directories will be created for you except for the input_path. This should point to your DICOMs, separated into folders by case like so:

    input_path
    └─── case1
        │─── *
    └─── case2
        │─── *
    └─── ...

Don't worry about preprocessing your dicoms, separating by scan type, or excluding non-cines. The pipeline should find which ones are cines and which ones aren't by checking key terms within the series descriptions. Check src/bivme/preprocessing/dicom/extract_cines.py for the list of key terms.

In [8]:
batch_ID = config['input_pp']['batch_ID'] # This will serve as your output folder name. Example: 'test'
analyst_id = config['input_pp']['analyst_id'] # Example: 'analyst1'
input_path = config['input_pp']['source'] # Path to the input DICOM folder
processed_path = config['input_pp']['processing'] # Path to the processed folder, where view predictions and segmentations will be stored. This will be created upon run time.
states_path = config['input_pp']['states'] # Path to the states folder, where the logs and view predictions will be stored for reference. This will be created upon run time.
output_path = config['output_pp']['output_directory'] # Path to the output folder, where GP files will be stored. This will be created upon run time.
plotting_path = processed_path # Path to the plotting folder, where the HTML file of plotted guidepoints will be stored. This will be created upon run time.

# Target path: src/bivme/preprocessing/dicom/models
# You can hardcode the model path here or set it as an environment variable.
cwd = os.getcwd()
print('Current working directory:', cwd)
MODEL_DIR = os.path.join(cwd,'preprocessing/dicom/models')
print('Model directory:', MODEL_DIR)

Current working directory: c:\Users\jdil469\Code\biv-me-dev\src\bivme
Model directory: c:\Users\jdil469\Code\biv-me-dev\src\bivme\preprocessing/dicom/models


## Step 0.1: Choose case

In [9]:
case = 'patient1' # Enter the case ID. This should be a subfolder within the input_path. 

In [10]:
# Set up batch specific folders
src = os.path.join(input_path)
dst = os.path.join(processed_path, batch_ID)
states = os.path.join(states_path, batch_ID)
output = os.path.join(output_path, batch_ID)
plotting = os.path.join(plotting_path, batch_ID)


case_src = os.path.join(src, case)
if not os.path.isdir(case_src):
    logger.error(f'Case {case} not found in source folder. Please check the case ID.')
    sys.exit()

os.makedirs(dst, exist_ok=True)
os.makedirs(states, exist_ok=True)
os.makedirs(output, exist_ok=True)
os.makedirs(plotting, exist_ok=True)

case_dst = os.path.join(dst, case)
case_output = os.path.join(output, case)

if os.path.exists(case_output):
    overwrite = config['output_pp']['overwrite']
    if overwrite:
        shutil.rmtree(case_dst)
        shutil.rmtree(case_output)
    else:
        print(f'This case has already been processed and you have set overwrite to false in the config file. Change the case ID or delete the existing folder {case_dst} before proceeding.')
        sys.exit()

elif os.path.exists(case_dst):
    overwrite = config['output_pp']['overwrite']
    if overwrite:
        shutil.rmtree(case_dst)
    else:
        print(f'This case has already been processed and you have set overwrite to false in the config file. Change the case ID or delete the existing folder {case_dst} before proceeding.')
        sys.exit()

In [11]:
# Create log file to record some details
states = os.path.join(states, case, analyst_id)
os.makedirs(states, exist_ok=True)

logger_id = logger.add(f'{states}/log_file_{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.log', level=log_level, format=log_format,
    colorize=False, backtrace=True,
    diagnose=True)

logger.info(f'Processing case: {case}')

start_time = time.time()

2025-08-29 16:08:10.972 | INFO     | __main__:<module>:9 - Processing case: patient1


## Step 0.2: Pre-preprocess dicoms
By default, the pipeline takes in only cardiac cine images in .dcm format. The pre-preprocessing reads in the raw dicoms and uses the series descriptions to infer which are cines and which aren't. The 'cine only' dicoms are saved to:

    processed_path   
    └───batch_ID
        └───processed-dicoms

In [12]:
logger.info(f'Finding cines...')
extract_cines(case_src, case_dst, logger)

case_src = os.path.join(case_dst, 'processed-dicoms') # Update source directory

logger.success(f'Pre-preprocessing complete. Cines extracted to {case_src}.')

2025-08-29 16:08:11.224 | INFO     | __main__:<module>:1 - Finding cines...
2025-08-29 16:08:16.811 | INFO     | bivme.preprocessing.dicom.extract_cines:extract_cines:45 - Found 275 images in the source directory
2025-08-29 16:08:16.812 | INFO     | bivme.preprocessing.dicom.extract_cines:extract_cines:46 - Extracted 275 which matched the inclusion criteria.
2025-08-29 16:08:21.667 | SUCCESS  | __main__:<module>:6 - Pre-preprocessing complete. Cines extracted to ../../example/processing\test\patient1\processed-dicoms.


## Step 1: View selection
View selection can be carried out in three main ways. 

The first option is the **'default'** option. This involves the use of two models - one using only dicom metadata and one using only image data. The metadata predictions are only broad categories (SAX, LAX, or Outflow) and are used to correct image-based predictions. This should be the most robust option, but takes longer to complete. 

The second option is the **'image-only'** option. This only uses image data. Use this if the metadata step is adding unnecessary time, or is not performing as expected.

The third option is the **'load'** option. This loads view predictions from the states folder. This is useful if view predictions have already been made, avoiding the need to rerun prediction from scratch. It also allows for you to make and save manual corrections to the selected views. 

On top of these options, you can also manually review and correct the selected views using a simple GUI. There are three options - **'manual'**, which always launches the GUI after view selection, **'adaptive'**, which only launches the GUI if there are low confidence predictions or missing views, or **'automatic'**, which does not launch the GUI and keeps the automatic predictions as they are.

#### Important note about required views
To proceed with fitting, you must have at least some SAX images and a 4ch image. This is because the fitting requires landmarks from the 4ch image to initialise the model (mitral valve, tricuspid valve, and apex point). The SAX images are required, along with at least one long axis image e.g. 4ch, to correct for breath-hold misalignment. Using the GUI on **'adaptive'** mode will make sure that you never miss a required view. 

In [13]:
option = 'default' # Either 'default', 'image-only', or 'load'. 
# 'default' will combine dicom metadata and image data to select the correct view. 'image-only' will only use the image data. 
# 'load' will load view predictions from the states folder, if view predictions have already been made.

correct_mode = 'manual' # Either 'manual', 'adaptive', or 'automatic'. 
# 'manual' will launch a GUI after completing view predictions to allow you to correct any mispredictions. 
# 'adaptive' will only launch a GUI if there are any poor predictions or missing views
# 'automatic' will not launch the GUI and will use the predictions as they are.

slice_info_df, num_phases, slice_mapping = select_views(case, case_src, case_dst, MODEL_DIR, states, option, correct_mode, logger)

logger.success(f'View selection complete.')
logger.info(f'Number of phases: {num_phases}')

2025-08-29 16:08:21.867 | INFO     | bivme.preprocessing.dicom.select_views:select_views:97 - Performing metadata-based view prediction...
2025-08-29 16:08:31.134 | SUCCESS  | bivme.preprocessing.dicom.select_views:select_views:100 - Metadata-based view prediction complete.
2025-08-29 16:08:31.134 | INFO     | bivme.preprocessing.dicom.select_views:select_views:103 - Performing image-based view prediction...
2025-08-29 16:08:42.956 | SUCCESS  | bivme.preprocessing.dicom.select_views:select_views:107 - Image-based view prediction complete.
2025-08-29 16:08:42.958 | INFO     | bivme.preprocessing.dicom.select_views:select_views:110 - Combining metadata and image-based view predictions...
2025-08-29 16:08:43.033 | WARNING  | bivme.preprocessing.dicom.select_views:select_views:137 - Low confidence for series 8001 with image based prediction (SAX)...
2025-08-29 16:08:43.056 | SUCCESS  | bivme.preprocessing.dicom.select_views:select_views:181 - View predictions for patient1:
2025-08-29 16:08

## Step 2: Segmentation
Using the view classifications from step 1, images will be written into nifti form and sent to the segmentation models to be automatically segmented. There is one segmentation model specific to each view (currently SAX, 2ch, 3ch, 4ch, or RVOT), so only images belonging to those classes will be used from now on. 

In [14]:
## Segmentation
seg_start_time = time.time()
segment_views(case_dst, MODEL_DIR, slice_info_df, logger) # Segmentation occurs on 2D+t (3D) images using 3D nnU-Net models
seg_end_time = time.time()

logger.success(f'Segmentation complete. Time taken: {seg_end_time-seg_start_time} seconds.')

2025-08-29 16:08:59.196 | INFO     | bivme.preprocessing.dicom.segment_views:segment_views:94 - Writing SAX images to nifti files...
2025-08-29 16:08:59.463 | INFO     | bivme.preprocessing.dicom.segment_views:segment_views:109 - Segmenting SAX images...


There are 5 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 5 cases that I would like to predict

Predicting SAX_3d_10001:
perform_everything_on_device: True


100%|██████████| 2/2 [00:03<00:00,  1.93s/it]


sending off prediction to background worker for resampling and export
done with SAX_3d_10001

Predicting SAX_3d_11001:
perform_everything_on_device: True


100%|██████████| 2/2 [00:01<00:00,  1.44it/s]


...

2025-08-29 16:11:28.829 | INFO     | bivme.preprocessing.dicom.segment_views:predict_view:62 - Done with 4ch
2025-08-29 16:11:28.838 | INFO     | bivme.preprocessing.dicom.segment_views:segment_views:94 - Writing RVOT images to nifti files...
2025-08-29 16:11:28.914 | INFO     | bivme.preprocessing.dicom.segment_views:segment_views:109 - Segmenting RVOT images...


There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting RVOT_3d_22001:
perform_everything_on_device: True


100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


sending off prediction to background worker for resampling and export
done with RVOT_3d_22001


2025-08-29 16:11:58.671 | INFO     | bivme.preprocessing.dicom.segment_views:predict_view:62 - Done with RVOT
2025-08-29 16:11:58.674 | SUCCESS  | __main__:<module>:6 - Segmentation complete. Time taken: 179.48266506195068 seconds.


## Step 2.1: Correct for mismatching phases (conditional)
Often, LAX and SAX series do not have matching number of phases. In that case, we need to resample segmentations to have the same number of phases. We use the SAX series as the reference for the 'right' number of phases.

In [15]:
## Resample segmentations if phases are not consistent between SAX and LAX views
correct_phase_mismatch(case_dst, slice_info_df, num_phases, logger)

2025-08-29 16:11:59.253 | SUCCESS  | bivme.preprocessing.dicom.correct_phase_mismatch:correct_phase_mismatch:27 - No phase mismatches found. No resampling required.


## Step 2.2: Review segmentations (optional)
Images and segmentations are stored here in nifti format (.nii.gz). 

    processed_path   
    └───batch_ID
        └───case


If needed, segmentations can be corrected here, and subsequent code executions will incorporate those changes. 3D Slicer or ITKSnap are good tools for correcting segmentations. We are working on a way to automatically load the images and segmentations into Slicer via a Python backend to make this review process easier. 

## Step 3: Generate contours
Contours are generated from the perimeters of the segmentation labels, and other key landmarks (mitral valve, tricuspid valve, aortic valve, pulmonary valve, rv inserts, and LV apex) are estimated from intersections of contours. 2D contours are transformed into 3D cartesian space using the affine constructed from the image position, orientation, and pixel spacing metadata in the DICOMs.

In [16]:
slice_dict = generate_contours(case_dst, slice_info_df, num_phases, logger)
logger.success(f'Guide points generated successfully.')

2025-08-29 16:11:59.549 | INFO     | bivme.preprocessing.dicom.generate_contours:generate_contours:10 - Generating contours for SAX slice 8001...
2025-08-29 16:11:59.810 | INFO     | bivme.preprocessing.dicom.generate_contours:generate_contours:10 - Generating contours for SAX slice 9001...
2025-08-29 16:12:00.217 | INFO     | bivme.preprocessing.dicom.generate_contours:generate_contours:10 - Generating contours for SAX slice 10001...
2025-08-29 16:12:00.526 | INFO     | bivme.preprocessing.dicom.generate_contours:generate_contours:10 - Generating contours for SAX slice 11001...
2025-08-29 16:12:00.741 | INFO     | bivme.preprocessing.dicom.generate_contours:generate_contours:10 - Generating contours for SAX slice 12001...
2025-08-29 16:12:00.877 | INFO     | bivme.preprocessing.dicom.generate_contours:generate_contours:10 - Generating contours for 2ch slice 17001...
2025-08-29 16:12:01.150 | INFO     | bivme.preprocessing.dicom.generate_contours:generate_contours:10 - Generating conto

## Step 4: Export guidepoints
Contours are then exported into a standardised form, called a GPFile. There is one of these files per frame. This is what the model fitting part of the code requires. There is also a SliceInfoFile which contains the affine information for each slice, which is necessary to allow for breath hold correction prior to model fitting. 

Temporal smoothing of landmark points is enabled by default, but can be disabled by changing the boolean below. 

In [17]:
smooth_landmarks = True # Set to True to smooth the landmarks, False to keep them as they are
gp_dir = os.path.join(output, case)

export_guidepoints(case_dst, gp_dir, slice_dict, slice_mapping, smooth_landmarks)
logger.success(f'Guide points exported successfully.')
logger.success(f'Case {case} complete.')
logger.info(f'Total time taken: {time.time()-start_time} seconds.')

2025-08-29 16:12:06.897 | SUCCESS  | __main__:<module>:5 - Guide points exported successfully.
2025-08-29 16:12:06.899 | SUCCESS  | __main__:<module>:6 - Case patient1 complete.
2025-08-29 16:12:06.901 | INFO     | __main__:<module>:7 - Total time taken: 235.92653131484985 seconds.


## Step 5 (Optional): Plot guidepoints
The code below will plot the guidepoints at each frame as html files, which can be viewed at:

    plotting_path   
    └───batch_ID
        └───case
            └───html

Here's a good place to find any issues before proceeding to model fitting. 

In [18]:
generate_plots_preprocessing = False # Set to True to generate plots for preprocessing, False to skip plotting
include_images = True # Set to True to include images in the HTML file, False to exclude them. Setting to False will speed up the plotting process, as images take a long time to plot and take up a lot of space in the HTML file.

if generate_plots_preprocessing:
    if include_images:
        image_path = os.path.join(case_dst, 'images')
    else:
        image_path = None

    generate_html(gp_dir, out_dir=plotting, gp_suffix='', si_suffix='', frames_to_fit=[], my_logger=logger, model_path=None, image_path=image_path)

    logger.info(f'Guidepoints plotted at {os.path.join(plotting,case,"html")}.')

# Start of model fitting

# Step 6: Model fitting
The model fitting takes in the GPFiles and SliceInfoFiles generated during preprocessing to generate biventricular meshes at each frame. An iterative linear least squares fit is applied, as described in Mauger, C., Gilbert, K., Suinesiaputra, A., Pontre, B., Omens, J., McCulloch, A., & Young, A. (2018, July). An iterative diffeomorphic algorithm for registration of subdivision surfaces: application to congenital heart disease. In 2018 40th Annual International Conference of the IEEE Engineering in Medicine and Biology Society (EMBC) (pp. 596-599). IEEE. DOI: 10.1109/EMBC.2018.8512394. 

Breath hold misalignment is corrected by the intersection of short and long axis contours. 

Meshes are generated in the output directory you provided in the config file. Mesh vertices are exported in .txt format by default, but can also be exported with faces in .vtk and .obj formats as well for visualisation and postprocessing purposes.

Unfortunately, there is no opportunity for intervention during this step. 

In [19]:
# Set input for fitting equal to the output of preprocessing
config["input_fitting"]["gp_directory"] = output

# Save a copy of the config file to the output folder
output_folder = Path(config["output_fitting"]["output_directory"])
output_folder.mkdir(parents=True, exist_ok=True)
shutil.copy(config_file, output_folder)

# Force overwrite to true
config["output_fitting"]["overwrite"] = True

# Set whether to generate plots for fitting
config["plotting"]["generate_plots_fitting"] = True
config["plotting"]["include_images"] = True # Set to True to include images in the HTML plots, False to exclude them. Setting to False will speed up the plotting process, as images take a long time to plot and take up a lot of space in the HTML file.

In [20]:
# Where Charlène's magic happens
run_fitting(case, config, logger)

2025-08-29 16:12:07.861 | INFO     | main:run_fitting:26 - Processing patient1
2025-08-29 16:12:07.869 | INFO     | bivme.fitting.perform_fit:perform_fitting:80 - case: patient1
2025-08-29 16:12:07.874 | INFO     | bivme.fitting.perform_fit:perform_fitting:88 - ED set to frame #0
2025-08-29 16:12:07.877 | INFO     | bivme.fitting.perform_fit:perform_fitting:105 - Shift measured only at ED frame
2025-08-29 16:12:10.384 | INFO     | bivme.fitting.perform_fit:perform_fitting:204 - Calculating pose and scale patient1...
2025-08-29 16:12:10.521 | INFO     | bivme.fitting.perform_fit:perform_fitting:209 - Fitting of patient1
2025-08-29 16:12:10.530 | INFO     | bivme.fitting.perform_fit:perform_fitting:222 - Processing frame #0
2025-08-29 16:12:11.070 | INFO     | bivme.fitting.diffeomorphic_fitting_utils:solve_least_squares_problem:277 -      Iteration 1 Weight 1000000000000.0	 ICF error 2.4073344137637562
2025-08-29 16:12:11.521 | INFO     | bivme.fitting.diffeomorphic_fitting_utils:solve_

2025-08-29 16:21:14.138 | INFO     | main:run_fitting:38 - Average residuals: 1.2333102084996408 for case patient1


## (Optional) Analysis and/or visualisation of models
You can visualise the fitted models frame-by-frame in your browser by opening an html in `output_directory\case\html`. Alternatively, if you have a mesh visualisation software, you can load in the .vtk or .obj files you generated to view the meshes beating through time. I recommend Paraview, as it is open source, and has nice camera controls for creating  videos of the meshes beating through time.     

We haven't added any analysis tools to this notebook yet. Refer to the main README on how to generate metrics from the biv-me models. 